In [7]:
import utils
import numpy as np
import pandas as pd

In [8]:
INPUT_PATH = "test_results/test_neurons_results_unsloth--Meta-Llama-3.1-8B-Instruct-bnb-4bit.csv"

neuron_results_df = pd.read_csv(INPUT_PATH)
neuron_results_df

,Unnamed: 0,layer,neuron,explainer,NeuronViewer,Original,Summary,Highlight,HighlightSummary,AVHS
0,0,0,286,unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit,words related to comparison.,"\n\ninstances of the word ""Compared"".",\n\nphrases expressing comparison or contrast ...,"\n\nwords containing the suffix ""-ed"" where a ...",\n\nwords that indicate similarities or contra...,"\n\nphrases that use ""Compar"" followed by a pr..."
1,1,10,1838,unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit,phrases describing positions or situations in...,\n\nwords related to people or dogs being in s...,\n\nphrases where a noun (person or entity) is...,\n\nwords or phrases where the subject finds o...,\n\nfirst person singular pronouns.,\n\nfirst or third person singular pronouns wh...
2,2,20,193,unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit,verbs indicating questioning or challenging be...,\n\ninstances where others are actively callin...,\n\nwords associated with social critique or d...,\n\nwords related to confrontation or direct i...,\n\nverbs related to confrontation or calling ...,\n\nverb phrases related to speaking out or ex...
3,3,30,1685,unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit,expressions of gratitude and agreeing to rece...,\n\nsubscription agreements or confirmation.,\n\nphrases indicating confirmation of subscri...,\n\nphrases related to subscription or confirm...,"\n\nphrases related to user interaction, speci...",\n\nwords or phrases associated with opt-in or...
4,4,40,431,unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit,"numbers related to time, dates, and measureme...",\n\nphrases or sentences related to time or date.,\n\ntime or date in 24-hour format.,\n\nwords and phrases related to time indicati...,"\n\ntime and clock-related tokens, including d...","\n\ntime or date specifications, such as combi..."


In [9]:
def get_embedding(text_to_embed):
	# Embed a line of text
	response = utils.embedding_with_backoff(
    	text_to_embed.strip()
	)
	# Extract the AI output embedding as a list of floats
	return response

def cosSim(A, B):
  A=np.array(A).flatten()
  B=np.array(B).flatten()
  return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))

In [10]:
modes = ["Original", "Summary", "Highlight", "HighlightSummary", "AVHS"]
AdaCS_results = {"layer":[], "neuron":[]}
for mode in modes:
    AdaCS_results[mode] = []

for i, row in neuron_results_df.iterrows():
    layer = row["layer"]
    neuron = row["neuron"]
    NV_Explan = row["NeuronViewer"]
    AdaCS_results["layer"].append(layer)
    AdaCS_results["neuron"].append(neuron)
    
    for mode in modes:
        prompt_Explan_Vector = get_embedding(row[mode])
        NV_Explan_Vector = get_embedding(NV_Explan)
        AdaCS_results[mode].append(cosSim(prompt_Explan_Vector, NV_Explan_Vector))

new_df = pd.DataFrame(AdaCS_results)
new_df

,layer,neuron,Original,Summary,Highlight,HighlightSummary,AVHS
0,0,286,0.842653,0.806121,0.574227,0.802254,0.817806
1,10,1838,0.674169,0.810747,0.743016,0.637503,0.632520
2,20,193,0.687970,0.714919,0.839472,0.769814,0.772463
3,30,1685,0.638621,0.683889,0.830753,0.617181,0.621379
4,40,431,0.741581,0.669452,0.746680,0.722635,0.835504


In [11]:
new_df.to_csv(INPUT_PATH.split(".")[0] + "_adaCS_unsloth--Meta-Llama-3.1-8B-Instruct-bnb-4bit.csv")

In [12]:
for mode in modes:
    avg = np.mean(new_df[mode])
    print(mode, avg)

Original 0.7169987
Summary 0.7370256
Highlight 0.74682957
HighlightSummary 0.7098774
AVHS 0.7359344
